In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

BASE_URL = "https://ideas.repec.org"
LIST_PAGE = "https://ideas.repec.org/s/cen/wpaper.html"
HEADERS = {"User-Agent": "FSRDC-Project-Bot/1.0"}

# Load researchers from All_Metadata
RESEARCHER_URL = "https://raw.githubusercontent.com/dingkaihua/fsrdc-external-census-projects/master/metadata/All_Metadata.csv"
df_meta = pd.read_csv(RESEARCHER_URL)
researcher_cols = [col for col in df_meta.columns if col.startswith("Researcher")]
researchers = set()
for col in researcher_cols:
    researchers.update(df_meta[col].dropna().str.strip())

normalized_researchers = {re.sub(r'[^\w\s]', '', name).strip().lower() for name in researchers}

def normalize_name(name):
    return re.sub(r'[^\w\s]', '', name).strip().lower()

def match_researcher(authors):
    if authors == "N/A": return ""
    for author in authors.split(";"):
        if normalize_name(author) in normalized_researchers:
            return author.strip()
    return ""

def clean_authors(authors):
    return [a.strip() for a in authors if a.strip() and not a.lower().startswith("registered")]

def find_flags(abstract):
    if not abstract or abstract == "N/A":
        return {
            "mention_acknowledgment": False,
            "mention_restricted_data": False,
            "mention_disclosure_review": False,
            "mention_rdc": False,
            "matched_dataset_terms": ""
        }

    abstract_lower = abstract.lower()

    ack_patterns = ["census bureau", "fsrdc", "federal statistical research data center"]
    restricted_data = ["census", "irs", "bea", "restricted microdata", "confidential data"]
    disclosure_patterns = ["disclosure review", "confidentiality review", "reviewed for disclosure"]
    rdc_patterns = ["michigan rdc", "texas rdc", "chicago rdc", "minnesota rdc"]

    datasets = [
        "annual survey of manufactures",
        "census of construction industries",
        "census of finance, insurance, and real estate",
        "american community survey",
        "current population survey",
        "medical expenditure panel survey",
        "integrated longitudinal business database"
    ]
    matched_datasets = [d for d in datasets if d in abstract_lower]

    return {
        "mention_acknowledgment": any(p in abstract_lower for p in ack_patterns),
        "mention_restricted_data": any(p in abstract_lower for p in restricted_data),
        "mention_disclosure_review": any(p in abstract_lower for p in disclosure_patterns),
        "mention_rdc": any(p in abstract_lower for p in rdc_patterns),
        "matched_dataset_terms": "; ".join(matched_datasets)
    }

def get_paper_links():
    resp = requests.get(LIST_PAGE, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")
    return [BASE_URL + a['href'] for a in soup.select("ul.paperlist li a") if a['href'].startswith("/p/cen/wpaper/")]

def scrape_repec(url):
    resp = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(resp.text, "html.parser")
    title = soup.find("h1").text.strip() if soup.find("h1") else "N/A"
    abstract = soup.find("div", id="abstract-body")
    abstract = abstract.text.strip() if abstract else "N/A"
    authors = []
    ul = soup.find("ul", id="authorlist")
    if ul:
        authors += [li.text for li in ul.find_all("li")]
    reg_div = soup.find("div", id="registered-authors")
    if reg_div:
        authors += [a.text for a in reg_div.find_all("a")]
    authors = "; ".join(clean_authors(authors)) if authors else "N/A"
    year = re.search(r'/([0-9]{2})-', url)
    year = "20" + year.group(1) if year else "N/A"
    return {"title": title, "abstract": abstract, "authors": authors, "year": year, "url": url}

def query_openalex(title):
    try:
        resp = requests.get("https://api.openalex.org/works", params={"search": title, "per_page": 1}, headers=HEADERS)
        data = resp.json()
        if data.get("results"):
            r = data["results"][0]
            return {
                "doi": r.get("doi", "N/A"),
                "publication_date": r.get("publication_date", "N/A"),
                "cited_by_count": r.get("cited_by_count", "N/A"),
                "affiliations": "; ".join(
                    aff["institution"]["display_name"]
                    for auth in r.get("authorships", [])
                    for aff in auth.get("institutions", [])
                    if aff.get("institution")
                ) or "N/A",
                "source_display_name": r.get("host_venue", {}).get("display_name", "N/A"),
                "type_crossref": r.get("type", "N/A"),
                "topics": "; ".join(t["display_name"] for t in r.get("concepts", []))
            }
    except:
        pass
    return {
        "doi": "N/A", "publication_date": "N/A", "cited_by_count": "N/A",
        "affiliations": "N/A", "source_display_name": "N/A", "type_crossref": "N/A", "topics": "N/A"
    }

results = []
for i, link in enumerate(get_paper_links()):
    print(f"[{i+1}] {link}")
    base = scrape_repec(link)
    flags = find_flags(base["abstract"])
    if not any([
        flags["mention_acknowledgment"], flags["mention_disclosure_review"],
        flags["mention_restricted_data"], flags["mention_rdc"],
        flags["matched_dataset_terms"]
    ]):
        print("skipped (no FSRDC relevance)")
        continue
    matched = match_researcher(base["authors"])
    if not matched:
        print("skipped (no researcher match)")
        continue
    enriched = query_openalex(base["title"])
    results.append({
        **base, **enriched, **flags,
        "researcher": matched
    })
    time.sleep(1.2)

# Export
cols = [
    "title", "doi", "abstract", "year", "publication_date", "cited_by_count",
    "authors", "affiliations", "topics", "source_display_name", "type_crossref",
    "researcher", "matched_dataset_terms",
    "mention_acknowledgment", "mention_restricted_data", "mention_disclosure_review",
    "mention_rdc"
]
pd.DataFrame(results)[cols].to_csv("web_scraping_full_output.csv", index=False)
print("\n Done! Saved to 'web_scraping_full_output.csv'")


[1] https://ideas.repec.org/p/cen/wpaper/25-19.html
skipped (no researcher match)
[2] https://ideas.repec.org/p/cen/wpaper/25-18.html
skipped (no FSRDC relevance)
[3] https://ideas.repec.org/p/cen/wpaper/25-16.html
skipped (no researcher match)
[4] https://ideas.repec.org/p/cen/wpaper/25-15.html
[5] https://ideas.repec.org/p/cen/wpaper/25-14.html
[6] https://ideas.repec.org/p/cen/wpaper/25-13.html
skipped (no researcher match)
[7] https://ideas.repec.org/p/cen/wpaper/25-12.html
skipped (no FSRDC relevance)
[8] https://ideas.repec.org/p/cen/wpaper/25-11.html
skipped (no FSRDC relevance)
[9] https://ideas.repec.org/p/cen/wpaper/25-10.html
skipped (no researcher match)
[10] https://ideas.repec.org/p/cen/wpaper/25-09.html
skipped (no FSRDC relevance)
[11] https://ideas.repec.org/p/cen/wpaper/25-08.html
skipped (no FSRDC relevance)
[12] https://ideas.repec.org/p/cen/wpaper/25-07.html
skipped (no researcher match)
[13] https://ideas.repec.org/p/cen/wpaper/25-06.html
[14] https://ideas.repec.

In [21]:
def test_normalize_name():
    assert normalize_name("John A. Smith") == "john a smith"
    assert normalize_name(" Jane-Doe ") == "janedoe"
    assert normalize_name("Dr. María-José, Jr.") == "dr maríajosé jr"
    print("test_normalize_name passed.")

def test_match_researcher_custom():
    # Mock a mini researcher set
    global normalized_researchers
    normalized_researchers = {"john doe", "jane smith", "amy lin"}

    assert match_researcher("John Doe; Emily Zhang") == "John Doe"
    assert match_researcher("Someone Else; Amy Lin") == "Amy Lin"
    assert match_researcher("Unknown; Another") == ""
    assert match_researcher("N/A") == ""
    print("test_match_researcher passed.")

def test_clean_authors():
    dirty_list = ["John Doe", "Registered:", "", "Jane Smith", "  "]
    cleaned = clean_authors(dirty_list)
    print(cleaned)
    assert cleaned == ["John Doe", "Jane Smith"]
    print("test_clean_authors passed.")

def test_find_flags():
    abstract = (
        "This study uses the Census Bureau's restricted microdata from the American Community Survey. "
        "All analysis was reviewed for disclosure. Data were accessed via the Michigan RDC."
    )
    flags = find_flags(abstract)
    assert flags["mention_acknowledgment"] is True
    assert flags["mention_restricted_data"] is True
    assert flags["mention_disclosure_review"] is True
    assert flags["mention_rdc"] is True
    assert "american community survey" in flags["matched_dataset_terms"].lower()

    empty = find_flags("This is unrelated to government datasets.")
    assert not any([empty[k] for k in empty if k != "matched_dataset_terms"])
    assert empty["matched_dataset_terms"] == ""
    print("test_find_flags passed.")

test_normalize_name()
test_match_researcher_custom()
test_clean_authors()
test_find_flags()
print("All tests passed!\n")

test_normalize_name passed.
test_match_researcher passed.
['John Doe', 'Jane Smith']
test_clean_authors passed.
test_find_flags passed.
All tests passed!

